# Grassmann Manifold

Author: Ketson R. M. dos Santos,  
Date: June 3rd, 2020   

This example shows how to use the UQpy Grassmann class to
* use the logarithimic map;
* use the exponential map;
* compute the Karcher mean.

Import the necessary libraries. Here we import standard libraries such as numpy and matplotlib, but also need to import the Grassmann class from UQpy implemented in the DimensionReduction module.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from UQpy.dimension_reduction.grassman.manifold_projections.SvdProjection import SvdProjection
import sys
import copy
from UQpy import OptimizationMethod, RiemannianDistance

from UQpy.dimension_reduction import Grassmann

Generate four random matrices with reduced rank corresponding to the different samples. The samples are stored in `matrices`.

In [ ]:
D1 = 6
r0 = 2  # rank sample 0
r1 = 3  # rank sample 1
r2 = 4  # rank sample 2
r3 = 3  # rank sample 2

np.random.seed(1111)  # For reproducibility.
# Solutions: original space.
Sol0 = np.dot(np.random.rand(D1, r0), np.random.rand(r0, D1))
Sol1 = np.dot(np.random.rand(D1, r1), np.random.rand(r1, D1))
Sol2 = np.dot(np.random.rand(D1, r2), np.random.rand(r2, D1))
Sol3 = np.dot(np.random.rand(D1, r3), np.random.rand(r3, D1))

# Creating a list of matrices.
matrices = [Sol0, Sol1, Sol2, Sol3]

# Plot the matrices
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
ax1.title.set_text('Matrix 0')
ax1.imshow(Sol0)
ax2.title.set_text('Matrix 1')
ax2.imshow(Sol1)
ax3.title.set_text('Matrix 2')
ax3.imshow(Sol2)
ax4.title.set_text('Matrix 3')
ax4.imshow(Sol3)
plt.show()

Instatiate the UQpy class Grassmann using an user defined optimizer to compute the Karcher mean. Further, distance is also given by an user defined function.

In [ ]:
manifold_projection = SvdProjection(matrices, p_planes_dimensions=sys.maxsize)

# Plot the points on the Grassmann manifold defined by the left singular eigenvectors.
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
ax1.title.set_text('Matrix 0')
ax1.imshow(manifold_projection.psi[0])
ax2.title.set_text('Matrix 1')
ax2.imshow(manifold_projection.psi[0])
ax3.title.set_text('Matrix 2')
ax3.imshow(manifold_projection.psi[0])
ax4.title.set_text('Matrix 3')
ax4.imshow(manifold_projection.psi[0])
plt.show()

Compute the Karcher mean.

In [ ]:
class UserOptimization(OptimizationMethod):
    def __init__(self, acceleration: bool = False,
                 error_tolerance: float = 1e-3,
                 max_iterations: int = 1000):
        self.max_iterations = max_iterations
        self.error_tolerance = error_tolerance
        self.acceleration = acceleration

    def optimize(self, data_points, distance):
        points_number = len(data_points)
        alpha = 0.5
        rank = []
        for i in range(points_number):
            rank.append(min(np.shape(data_points[i])))

        from UQpy.dimension_reduction.grassman.Grassman import Grassmann
        max_rank = max(rank)
        fmean = []
        for i in range(points_number):
            fmean.append(Grassmann.frechet_variance(data_points[i], data_points, distance))

        index_0 = fmean.index(min(fmean))
        mean_element = data_points[index_0].tolist()

        avg_gamma = np.zeros([np.shape(data_points[0])[0], np.shape(data_points[0])[1]])

        counter_iteration = 0

        l = 0
        avg = []
        _gamma = []
        if self.acceleration:
            _gamma = Grassmann.log_map(points_grassmann=data_points,
                                       reference_point=np.asarray(mean_element))

            avg_gamma.fill(0)
            for i in range(points_number):
                avg_gamma += _gamma[i] / points_number
            avg.append(avg_gamma)

        # Main loop
        while counter_iteration <= self.max_iterations:
            _gamma = Grassmann.log_map(points_grassmann=data_points,
                                       reference_point=np.asarray(mean_element))
            avg_gamma.fill(0)

            for i in range(points_number):
                avg_gamma += _gamma[i] / points_number

            test_0 = np.linalg.norm(avg_gamma, 'fro')
            if test_0 < self.error_tolerance and counter_iteration == 0:
                break

            # Nesterov: Accelerated Gradient Descent
            if self.acceleration:
                avg.append(avg_gamma)
                l0 = l
                l1 = 0.5 * (1 + np.sqrt(1 + 4 * l * l))
                ls = (1 - l0) / l1
                step = (1 - ls) * avg[counter_iteration + 1] + ls * avg[counter_iteration]
                l = copy.copy(l1)
            else:
                step = alpha * avg_gamma

            x = Grassmann.exp_map(points_tangent=[step], reference_point=np.asarray(mean_element))

            test_1 = np.linalg.norm(x[0] - mean_element, 'fro')

            if test_1 < self.error_tolerance:
                break

            mean_element = []
            mean_element = x[0]

            counter_iteration += 1

        # return the Karcher mean.
        return mean_element

class UserDistance(RiemannianDistance):

    def compute_distance(self, x0, x1):
        if not isinstance(x0, list) and not isinstance(x0, np.ndarray):
            raise TypeError('UQpy: x0 must be either list or numpy.ndarray.')
        else:
            x0 = np.array(x0)

        if not isinstance(x1, list) and not isinstance(x1, np.ndarray):
            raise TypeError('UQpy: x1 must be either list or numpy.ndarray.')
        else:
            x1 = np.array(x1)

        l = min(np.shape(x0))
        k = min(np.shape(x1))
        rank = min(l, k)

        r = np.dot(x0.T, x1)
        # (ui, si, vi) = svd(r, rank)

        ui, si, vi = np.linalg.svd(r, full_matrices=True, hermitian=False)  # Compute the SVD of matrix
        si = np.diag(si)  # Transform the array si into a diagonal matrix containing the singular values
        vi = vi.T  # Transpose of vi

        u = ui[:, :rank]
        s = si[:rank, :rank]
        v = vi[:, :rank]

        index = np.where(si > 1)
        si[index] = 1.0
        theta = np.arccos(si)
        theta = np.sin(theta / 2) ** 2
        distance = np.sqrt(abs(k - l) + 2 * np.sum(theta))

        return distance

optimization_method= UserOptimization(acceleration=True, error_tolerance=1e-4,
                                      max_iterations=1000)
distance = UserDistance()
karcher_psi = Grassmann.karcher_mean(points_grassmann=manifold_projection.psi,
                                     p_planes_dimensions=manifold_projection.p_planes_dimensions,
                                     optimization_method=optimization_method,
                                     distance=distance)
karcher_phi = Grassmann.karcher_mean(points_grassmann=manifold_projection.phi,
                                     p_planes_dimensions=manifold_projection.p_planes_dimensions,
                                     optimization_method=optimization_method,
                                     distance=distance)

Project $\Psi$, the left singular eigenvectors, on the tangent space centered at the Karcher mean.

In [ ]:
points_tangent = Grassmann.log_map(points_grassmann=manifold_projection.psi,
                                   reference_point=karcher_psi)

print(points_tangent[0])
print(points_tangent[1])
print(points_tangent[2])
print(points_tangent[3])

# Plot the matrices
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
ax1.title.set_text('Matrix 0')
ax1.imshow(points_tangent[0])
ax2.title.set_text('Matrix 1')
ax2.imshow(points_tangent[1])
ax3.title.set_text('Matrix 2')
ax3.imshow(points_tangent[2])
ax4.title.set_text('Matrix 3')
ax4.imshow(points_tangent[3])
plt.show()

Map the points back to the Grassmann manifold.

In [ ]:
points_grassmann = Grassmann.exp_map(points_tangent=points_tangent,
                                     reference_point=manifold_projection.psi[0])

print(points_grassmann[0])
print(points_grassmann[1])
print(points_grassmann[2])
print(points_grassmann[3])

# Plot the matrices
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
ax1.title.set_text('Matrix 0')
ax1.imshow(points_grassmann[0])
ax2.title.set_text('Matrix 1')
ax2.imshow(points_grassmann[1])
ax3.title.set_text('Matrix 2')
ax3.imshow(points_grassmann[2])
ax4.title.set_text('Matrix 3')
ax4.imshow(points_grassmann[3])
plt.show()